In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
# from Model_Build.Functions import *

In [2]:
file_name = 'In_Season/Data/opening_day_rosters.pickle'
with open(file_name, 'rb') as f:
    opening_day_rosters = pickle.load(f)

In [3]:
file_name = 'Model_Build/Data/predict_vorp_regression.pickle'
with open(file_name, 'rb') as f:
    model = pickle.load(f)

In [4]:
def name_adjustment(x):
    x = x.split(' ')
    x = x[0] + ' ' + x[1]      
    return x
file_name = 'In_Season/Data/vorp_predictive_data.csv'
prior_year_predictors = pd.read_csv(file_name, index_col = 0)
prior_year_predictors['Player'] = prior_year_predictors.Player.apply(unidecode.unidecode)
prior_year_predictors['Player'] = prior_year_predictors.Player.apply(name_adjustment)
prior_year_predictors['Player'] = prior_year_predictors.Player.str.replace('.', '')
prior_year_predictors['Player'] = prior_year_predictors.Player.str.replace("'", '')
prior_year_predictors['Player'] = prior_year_predictors.Player.str.lower()

In [5]:
missed_players = list()
vorps = pd.DataFrame(columns = ['Team', 'VORP_projection'])
for key, value in opening_day_rosters.items():
    team = key
    roster = value
    team_vorp = 0
    for index, row in roster.iterrows():
        player = row.Player
        service = row.YOS
        if service > 0:
            player = player.replace('.', '')
            player = player.replace(',', '')
            player = player.replace("'", '')
            player = player.split(' ')
            player = player[0] + ' ' + player[1]
            player = player.lower()
            
            predictors = prior_year_predictors[prior_year_predictors.Player == player]
            if len(predictors) > 0:
                predictors = predictors[['Age', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
                       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
                       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP_Prior_Year']]
                vorp_prediction = model.predict(predictors)[0]
                team_vorp += vorp_prediction
                
            if len(predictors) == 0:
                
                missed_players.append(team + ': ' + player)
            
    series = pd.Series([team, team_vorp], index = vorps.columns)
    vorps = vorps.append(series, ignore_index = True)


In [6]:
missed_players

['Boston Celtics: enes kanter',
 'Brooklyn Nets: nicolas claxton',
 'Golden State Warriors: klay thompson',
 'Houston Rockets: kj martin',
 'Los Angeles Clippers: jayden scrubb',
 'Miami Heat: omer yurtseven',
 'New Orleans Pelicans: nicolas claxton',
 'New York Knicks: wayne selden',
 'Oklahoma City Thunder: mohamed bamba',
 'Oklahoma City Thunder: jonathan isaac',
 'Oklahoma City Thunder: moe wagner',
 'Sacramento Kings: zach collins',
 'Toronto Raptors: ogugua anunoby',
 'Toronto Raptors: sam dekker',
 'Washington Wizards: raulzinho neto']

In [10]:
prior_year_predictors[-150:-100]

,Player,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP_Prior_Year
390,naz reid,21,79.722222,1534.083333,18.200000,0.599000,0.299000,0.286000,6.400000,20.000000,...,9.500000,22.500000,2.163889,1.480556,3.644444,0.113000,-0.113889,-0.227778,-0.227778,0.683333
391,nemanja bjelica,32,84.277778,1355.277778,12.966667,0.553000,0.469333,0.245333,5.033333,17.933333,...,12.166667,17.866667,1.138889,0.683333,2.050000,0.081667,-5.125000,-1.936111,-6.947222,-0.113889
392,nerlens noel,26,72.888889,1761.861111,14.800000,0.638000,0.013000,0.314000,10.100000,18.400000,...,20.400000,9.200000,1.822222,4.100000,5.922222,0.162000,-2.733333,4.327778,1.594444,1.480556
393,nic claxton,21,36.444444,676.500000,16.900000,0.612000,0.034000,0.441000,8.400000,21.800000,...,9.400000,14.000000,1.138889,1.025000,2.163889,0.153000,-1.138889,1.594444,0.455556,0.455556
394,nick richards,23,20.500000,71.750000,5.500000,0.542000,0.111000,1.222000,6.800000,12.200000,...,17.800000,11.500000,0.000000,0.000000,0.000000,0.026000,-6.719444,-2.391667,-9.111111,-0.113889
395,nickeil alexander-walker,22,52.388889,1146.861111,12.500000,0.522000,0.478000,0.144000,1.400000,14.100000,...,12.400000,23.200000,-0.341667,1.138889,0.797222,0.035000,-1.594444,0.113889,-1.480556,0.227778
396,nico mannion,19,34.166667,414.555556,9.300000,0.487000,0.526000,0.246000,1.800000,11.500000,...,19.700000,18.500000,-0.227778,0.455556,0.227778,0.029000,-3.872222,-0.569444,-4.441667,-0.227778
397,nicolas batum,32,76.305556,2089.861111,12.900000,0.617000,0.660000,0.141000,3.200000,15.900000,...,10.800000,11.800000,3.075000,2.619444,5.694444,0.132000,0.683333,1.708333,2.391667,2.163889
398,nicolo melli,30,102.500000,1289.222222,7.700000,0.431000,0.605000,0.172000,5.600000,18.100000,...,7.566667,12.500000,0.227778,1.025000,1.252778,0.044667,-12.186111,0.341667,-12.072222,-0.455556
399,nikola jokic,25,82.000000,2833.555556,31.300000,0.647000,0.183000,0.305000,9.400000,26.100000,...,13.100000,29.600000,13.894444,3.872222,17.766667,0.301000,10.363889,3.416667,13.780556,10.022222


In [65]:
i = 2
team = 'Brooklyn-Nets'
link = f'https://basketball.realgm.com/nba/teams/{team}/{str(i+1)}/Rosters/Opening_Day'
tables = pd.read_html(link)
tables[-10]

,Player,Pos,HT,WT,Birth Date,Age,Pre-Draft Team,YOS,Draft Status
0,LaMelo Ball,SG,6-7,185,"Aug 22, 2001",20,Illawarra (Australia),1,"2020 1st Rnd, #3, Charlotte Hornets"
1,James Bouknight,SG,6-4,185,"Sep 18, 2000",21,Connecticut,0,"2021 1st Rnd, #11, Charlotte Hornets"
2,Miles Bridges,SF,6-7,225,"Mar 21, 1998",23,Michigan State,3,"2018 1st Rnd, #12, Los Angeles Clippers"
3,"Vernon Carey, Jr.",PF,6-9,258,"Feb 25, 2001",20,Duke,1,"2020 2nd Rnd, #32, Charlotte Hornets"
4,Gordon Hayward,SF,6-7,230,"Mar 23, 1990",31,Butler,11,"2010 1st Rnd, #9, Utah Jazz"
5,Kai Jones,PF,6-10,225,"Jan 19, 2001",20,Texas,0,"2021 1st Rnd, #19, New York Knicks"
6,Arnoldas Kulboka,SF,6-10,225,"Jan 4, 1998",23,Orlandina Basket (Italy),0,"2018 2nd Rnd, #55, Charlotte Hornets"
7,Scottie Lewis,SF,6-4,190,"Mar 12, 2000",21,Florida,0,"2021 2nd Rnd, #56, Charlotte Hornets"
8,Cody Martin,SF,6-6,200,"Sep 28, 1995",26,Nevada,2,"2019 2nd Rnd, #36, Charlotte Hornets"
9,Jalen McDaniels,PF,6-9,212,"Jan 31, 1998",23,San Diego State,2,"2019 2nd Rnd, #52, Charlotte Hornets"
